In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [1]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/mlm', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/modules']


2024-06-04 18:47:29.284471: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-04 18:47:29.470461: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 18:47:30.205483: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
from datasets import load_from_disk,concatenate_datasets


adapter_name="mlm_inv_S"
dataset = load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/datasets")



In [6]:
reload(config)

    
filtered_target = dataset['train'].filter(lambda example: example['genre'] == "slate").shuffle(seed=42)
#filtered_target = shuffled_filtered_target.train_test_split(test_size=0.1)


filtered_test_target = dataset['validation_matched'].filter(lambda example: example['genre'] == "slate")
train_target = filtered_target
test_target = filtered_test_target
train_target[1]
 

{'promptID': 15068,
 'pairID': '15068c',
 'premise': 'I am not gay.',
 'premise_binary_parse': '( I ( ( ( am not ) gay ) . ) )',
 'premise_parse': '(ROOT (S (NP (PRP I)) (VP (VBP am) (RB not) (ADJP (JJ gay))) (. .)))',
 'hypothesis': 'I am gay.',
 'hypothesis_binary_parse': '( I ( ( am gay ) . ) )',
 'hypothesis_parse': '(ROOT (S (NP (PRP I)) (VP (VBP am) (ADJP (JJ gay))) (. .)))',
 'genre': 'slate',
 'label': 2}

In [7]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
init(model)
reload(fn)
fn.print_trainable_parameters(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


trainable params: 109514298 || all params: 109514298 || trainable%: 100.0


In [8]:
from adapters import ConfigUnion, LoRAConfig, PrefixTuningConfig, SeqBnConfig,SeqBnInvConfig,AdapterConfig,LoRAConfig


adapter_config = SeqBnInvConfig(reduction_factor=2)

model.add_adapter(adapter_name, config=adapter_config)

model.train_adapter([adapter_name])
model.active_adapters = adapter_name
print(model.adapter_summary())
fn.print_trainable_parameters(model)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
mlm_inv_S                bottleneck        7,387,776       6.785       1       1
--------------------------------------------------------------------------------
Full model                               108,891,648     100.000               0
trainable params: 7979904 || all params: 116902074 || trainable%: 6.82614407679371


In [9]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(train_target,tokenizer)
tokenized_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 77306
})

In [10]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset[444:470]

results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [11]:
# we group texts and chunk them
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 23975
})

In [12]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] not everyone in america goes to a big public high school, but everyone goes to a high school [MASK] [MASK] a hierarchy [MASK] popularity and cl [MASK] private [SEP] everyone has big public schools here. [SEP] [CLS] [MASK] am not gay [MASK] [SEP] i am gay. [SEP] [CLS] the word that is used around dougador'operator [MASK] [SEP] people only choose to say the word'operator [MASK] when talking to doug [SEP] [CLS] however [MASK] most college - bound colorado students take a different college entrance exam, making the proliferation an [MASK] measure of school quality [MASK] [SEP] some students take different college entrance exams. [SEP] [CLS] their issuing of [MASK] - term buy or short - term hold'

'>>> recommendations obviously intensifies pressure on companies to meet and beat earnings expectations jacksonville all costs. [SEP] most companies will meet their earning [MASK]. [SEP] [CLS] well, my wife nearly fainted to see me [MASK]igated in public, [MASK] less on [MASK] interne

In [13]:

downsampled_dataset = lm_datasets.train_test_split(
    test_size=0.1, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 21577
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2398
    })
})

In [14]:

reload(fn)
trainer = fn.train_mlm_model(model,adapter_name,data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['test'])

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/75 [00:00<?, ?it/s]

>>> Perplexity: 16.83


In [16]:
trainer.train()

  0%|          | 0/13500 [00:00<?, ?it/s]

{'loss': 2.4052, 'grad_norm': 2.0269875526428223, 'learning_rate': 9.866153846153846e-05, 'epoch': 1.0}


  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 2.052076578140259, 'eval_runtime': 7.4405, 'eval_samples_per_second': 322.289, 'eval_steps_per_second': 10.08, 'epoch': 1.0}
{'loss': 2.1259, 'grad_norm': 1.993073582649231, 'learning_rate': 9.347692307692308e-05, 'epoch': 2.0}


  0%|          | 0/75 [00:00<?, ?it/s]

...

  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 1.7776027917861938, 'eval_runtime': 7.4777, 'eval_samples_per_second': 320.685, 'eval_steps_per_second': 10.03, 'epoch': 19.0}


Overwriting existing adapter 'mlm_inv_S'.


{'train_runtime': 2918.271, 'train_samples_per_second': 147.875, 'train_steps_per_second': 4.626, 'train_loss': 1.9469223748220106, 'epoch': 19.0}


TrainOutput(global_step=12825, training_loss=1.9469223748220106, metrics={'train_runtime': 2918.271, 'train_samples_per_second': 147.875, 'train_steps_per_second': 4.626, 'train_loss': 1.9469223748220106, 'epoch': 19.0})

In [17]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/75 [00:00<?, ?it/s]

>>> Perplexity: 5.87


In [18]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)


: 

#### This is a report of mlm for goverment domain using whole genre as target and then splitt after tokenization with seed 42. I used computer metrix for preplexity and early stopping 
The results before:
>>> Perplexity: 16.83

The results after:
>>> Perplexity: 5.87